### environmental data used in this research (data/environmental_data) are downloaded from TerraClimete 
https://www.climatologylab.org/terraclimate.html
#download data from TerraClimete 
https://www.climatologylab.org/terraclimate.html

---
### there are different ways how to it, I first made a bash file in the website (https://www.climatologylab.org/wget-terraclimate.html) for  years of interest. then i run the bash in my laptop (see bash file in data/terraclimatedata/  terraclimate_wget-2.sh). 
then following R script used to extracted the "sampling locations" that i needs Latitude and longitude of locations are needed)

### name and scales of environmental factors: https://www.climatologylab.org/terraclimate-variables.html
aet (Actual Evapotranspiration, monthly total), units = mm

def (Climate Water Deficit, monthly total), units = mm

pet (Potential evapotranspiration, monthly total), units = mm
ppt (Precipitation, monthly total), units = mm
q (Runoff, monthly total), units = mm
soil (Soil Moisture, total column - at end of month), units = mm
srad (Downward surface shortwave radiation), units = W/m2
swe (Snow water equivalent - at end of month), units = mm
tmax (Max Temperature, average for month), units = C
tmin (Min Temperature, average for month), units = C
vap (Vapor pressure, average for month), units  = kPa
ws (Wind speed, average for month), units = m/s
vpd (Vapor Pressure Deficit, average for month), units = kpa
PDSI (Palmer Drought Severity Index, at end of month), units = unitless

---
### extract data for all the moth of the years from 2014 to 2022 of the sampling sites(6 sites , by latitude and longitude , then subset for the month of the sampling)

## map of sampling sites

Here are the sites with map links, 
JUG 
https://goo.gl/maps/NjyxPrgdRwKtzqL96
48°33'22.5"N 9°08'07.0"E
Hofgut Einsiedel 11, 72138 Kirchentellinsfurt

K6911
https://goo.gl/maps/gPJsXQ6m52aurHEq7

PFN
48°33'39.3"N 9°06'32.4"E
48.560928, 9.108996
https://goo.gl/maps/ECAAK3XK95ewzQJU9

ERG
https://goo.gl/maps/apwT4SyMawnWN3cK9

EYACH
https://goo.gl/maps/zdvBCtkxx4JkaDWx7

Wendelsheim
https://goo.gl/maps/wgfvWpAsDaTFkzcP7


In [14]:
#setworking directory to AbioticAraMicrobe/data/terraclimatedata
setwd('../data/terraclimatedata')

In [14]:
library(raster)
# read all the environmental files in .nc format for years 2014-2021(8 years * 14 environmentalfactors = 112)
# (later we only used until 2019)
files <- list.files(path = "." , pattern = "*.nc" , full.names = TRUE) #nc files
site <- c("JUG", "K69", "PFN", "ERG" ,"EY" , "WH" ) #locations
lon <- c(9.135278,  9.1008673, 9.108996, 8.809256 ,8.7834256 , 8.9350829 ) # longitudes of locations
lat <- c(48.556250,  48.5453195, 48.560928, 48.495249 , 48.4462842, 48.5063677) # Latitudes of locations

###next lines take around 30 min to read 112 .nc files do it once and save the file and read it every time that you need

all_data <- data.frame()
for (i in files){
    file <- stack(i)  
    points <- data.frame(site, lon, lat , row.names="site")
    ##Extract data from RasterLayer
    extract_points <- data.frame(extract(file, points))
    colnames(extract_points) = c(1:12) #months of year
    extract_points$site <- site
    extract_points$file_info <- i 
    all_data <- rbind(all_data , extract_points)
}
write.table(all_data , "all_data.txt" , sep = "\t" , quote = FALSE , row.names = FALSE)

In [159]:
head(all_data) # file contains 12 columns of months of years (numbers shows values of environmental factors) , site column and file info shows each environmental factor per year

,1,2,3,4,5,6,7,8,9,10,11,12,site,file_info
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,12.9,23.8,38.1,54.1,68.3,50.7,104.3,89.3,52.9,34.5,17.3,18.5,JUG,./TerraClimate_aet_2014.nc
2,13.1,23.8,38.2,55.3,69.6,51.5,103.9,88.6,52.9,34.8,17.5,18.3,K69,./TerraClimate_aet_2014.nc
3,13.1,23.8,38.2,55.3,69.6,51.5,103.9,88.6,52.9,34.8,17.5,18.3,PFN,./TerraClimate_aet_2014.nc
4,11.8,22.3,38.8,58.3,72.5,52.0,103.9,90.8,54.0,35.8,16.9,17.2,ERG,./TerraClimate_aet_2014.nc
5,12.0,21.9,37.9,57.1,70.7,50.9,103.7,89.1,52.9,35.4,17.0,17.3,EY,./TerraClimate_aet_2014.nc
6,12.3,23.6,38.7,56.1,68.9,50.9,104.4,91.1,53.5,35.6,16.8,18.6,WH,./TerraClimate_aet_2014.nc


In [50]:
#open the  saved file , and take dates that are needed (a month befor sampling)
all_data <- read.table("all_data.txt" , header = TRUE)
head(all_data)
colnames(all_data)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,site,file_info
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,12.9,23.8,38.1,54.1,68.3,50.7,104.3,89.3,52.9,34.5,17.3,18.5,JUG,./TerraClimate_aet_2014.nc
2,13.1,23.8,38.2,55.3,69.6,51.5,103.9,88.6,52.9,34.8,17.5,18.3,K69,./TerraClimate_aet_2014.nc
3,13.1,23.8,38.2,55.3,69.6,51.5,103.9,88.6,52.9,34.8,17.5,18.3,PFN,./TerraClimate_aet_2014.nc
4,11.8,22.3,38.8,58.3,72.5,52.0,103.9,90.8,54.0,35.8,16.9,17.2,ERG,./TerraClimate_aet_2014.nc
5,12.0,21.9,37.9,57.1,70.7,50.9,103.7,89.1,52.9,35.4,17.0,17.3,EY,./TerraClimate_aet_2014.nc
6,12.3,23.6,38.7,56.1,68.9,50.9,104.4,91.1,53.5,35.6,16.8,18.6,WH,./TerraClimate_aet_2014.nc


[1] "X1"        "X2"        "X3"        "X4"        "X5"        "X6"       
 [7] "X7"        "X8"        "X9"        "X10"       "X11"       "X12"      
[13] "site"      "file_info"

In [51]:
#change column names
colnames(all_data) = c("Jan_Spring" , "Feb_Spring" , "Mar_Spring" , "Apr_Spring" , "May_Spring" , "Jun_Spring" ,"July_Spring" , "Agu_Spring" ,"Sep_Fall" , "Oct_Fall" , "Nov_Fall" , "Dec_Fall" , "Site" , "File_info")

In [67]:
library(tidyr)
melt_env_data <- melt(all_data, id = c("Site","File_info"))
head(melt_env_data)
melt_env_data <- separate(data = melt_env_data , col = File_info , into = c("databace" , "env_factor" , "Year") , sep = "_" , remove = FALSE)
melt_env_data$Year <- gsub(".nc" ,"" , melt_env_data$Year)
melt_env_data$m_s_y <- paste(melt_env_data$variable , melt_env_data$Year , sep = "_")
head(melt_env_data)

#subset following month from melt_env_data

samplingdate <- read.table("samplingdate.txt" , header = TRUE)
samplingdate
#note for now we are taking the month of sampling for thoes month that sampling are done after 15th of the month and a month before for thoes that sampling down befor 15th of the month.
samplingdate$month_to_take_fromEnvData <- c('Mar_Spring_2014' , 'Nov_Fall_2014' ,'Feb_Spring_2015' ,'Nov_Fall_2015', 'Mar_Spring_2016' , 'Nov_Fall_2016' , 'Mar_Spring_2017' , 'Nov_Fall_2017' , 'Mar_Spring_2018' , 'Nov_Fall_2018' , 'Mar_Spring_2019')
samplingdate

env_data <- merge(samplingdate ,melt_env_data , by.x = "month_to_take_fromEnvData" , by.y = "m_s_y")
dim(env_data)
head(env_data)
#14 * 11 * 6 (#environmentalFactors * timePoints * sites)

,Site,File_info,variable,value
,<chr>,<chr>,<fct>,<dbl>
1,JUG,./TerraClimate_aet_2014.nc,Jan_Spring,12.9
2,K69,./TerraClimate_aet_2014.nc,Jan_Spring,13.1
3,PFN,./TerraClimate_aet_2014.nc,Jan_Spring,13.1
4,ERG,./TerraClimate_aet_2014.nc,Jan_Spring,11.8
5,EY,./TerraClimate_aet_2014.nc,Jan_Spring,12.0
6,WH,./TerraClimate_aet_2014.nc,Jan_Spring,12.3


,Site,File_info,databace,env_factor,Year,variable,value,m_s_y
,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>,<chr>
1,JUG,./TerraClimate_aet_2014.nc,./TerraClimate,aet,2014,Jan_Spring,12.9,Jan_Spring_2014
2,K69,./TerraClimate_aet_2014.nc,./TerraClimate,aet,2014,Jan_Spring,13.1,Jan_Spring_2014
3,PFN,./TerraClimate_aet_2014.nc,./TerraClimate,aet,2014,Jan_Spring,13.1,Jan_Spring_2014
4,ERG,./TerraClimate_aet_2014.nc,./TerraClimate,aet,2014,Jan_Spring,11.8,Jan_Spring_2014
5,EY,./TerraClimate_aet_2014.nc,./TerraClimate,aet,2014,Jan_Spring,12.0,Jan_Spring_2014
6,WH,./TerraClimate_aet_2014.nc,./TerraClimate,aet,2014,Jan_Spring,12.3,Jan_Spring_2014


dayofsampling,Month,year
<chr>,<chr>,<int>
2,apr,2014
3,Dec,2014
18,Feb,2015
1,dec,2015
27,mar,2016
5,dec,2016
20_21,mar,2017
26,nov,2017
4_5,apr,2018


dayofsampling,Month,year,month_to_take_fromEnvData
<chr>,<chr>,<int>,<chr>
2,apr,2014,Mar_Spring_2014
3,Dec,2014,Nov_Fall_2014
18,Feb,2015,Feb_Spring_2015
1,dec,2015,Nov_Fall_2015
27,mar,2016,Mar_Spring_2016
5,dec,2016,Nov_Fall_2016
20_21,mar,2017,Mar_Spring_2017
26,nov,2017,Nov_Fall_2017
4_5,apr,2018,Mar_Spring_2018


[1] 924  11

,month_to_take_fromEnvData,dayofsampling,Month,year,Site,File_info,databace,env_factor,Year,variable,value
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>
1,Feb_Spring_2015,18,Feb,2015,PFN,./TerraClimate_ppt_2015.nc,./TerraClimate,ppt,2015,Feb_Spring,21.20
2,Feb_Spring_2015,18,Feb,2015,K69,./TerraClimate_vpd_2015.nc,./TerraClimate,vpd,2015,Feb_Spring,0.14
3,Feb_Spring_2015,18,Feb,2015,ERG,./TerraClimate_ppt_2015.nc,./TerraClimate,ppt,2015,Feb_Spring,30.30
4,Feb_Spring_2015,18,Feb,2015,EY,./TerraClimate_PDSI_2015.nc,./TerraClimate,PDSI,2015,Feb_Spring,1.66
5,Feb_Spring_2015,18,Feb,2015,WH,./TerraClimate_soil_2015.nc,./TerraClimate,soil,2015,Feb_Spring,61.40
6,Feb_Spring_2015,18,Feb,2015,JUG,./TerraClimate_vpd_2015.nc,./TerraClimate,vpd,2015,Feb_Spring,0.14


In [68]:
length(unique(melt_env_data$env_factor))

[1] 14

In [71]:
#final table
write.table(env_data , "terraclimatedata/subset_envData.txt" , sep = "\t" , quote = FALSE , row.names = FALSE )